In [1]:
CHUNK_SIZE_MB = 100

In [2]:
from pathlib import Path
from dotenv import load_dotenv
import os
import vaex

In [3]:
file_path = f"../../Data/s.csv"
df = vaex.from_csv(file_path, convert=True, chunk_size=5_000_000, copy_index=False)
size = round(Path(file_path).stat().st_size / 1024 / 1024 / 1024, 3)

In [4]:
load_dotenv()

True

In [5]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region = os.getenv("AWS_REGION")
bucket_id = os.getenv("AWS_S3_BUCKET_ID")

In [11]:
size_mb = size * 1024
n_chunks = int((size_mb // CHUNK_SIZE_MB) + 1) if (size_mb % CHUNK_SIZE_MB) != 0 else int(size_mb // CHUNK_SIZE_MB)
chunk_size = int(df.count()) // n_chunks
file_prefix = "output_chunk-"
file_path = f"s3://{bucket_id}/" + "sample.parquet"


In [12]:
df.export_many(
    file_path,
    chunk_size=chunk_size,
    fs_options={
        "access_key": access_key,
        "secret_key": secret_access_key,
        "region": region
    }
)

In [25]:
df_open = vaex.open_many("s3://jfhuete-pycones2021/sample-**.parquet")

In [26]:
df_open.count()

array(100000)